# Confusion matrix for CocoAPI

The "main_Evaluate_Object_Detection.ipynb" uses the CocoApi to evaluate the model performance, e.g., AP50. But that code does not include the output of confusion matrix (e.g., TP, FP, and FN).

This script provides the output of confusion matrix using **FiftyOne** tool

FiftyOne’s implementation of COCO-style evaluation matches the reference implementation available via pycocotools (https://github.com/cocodataset/cocoapi).

Refer to: 
https://docs.voxel51.com/integrations/coco.html#loading-coco-formatted-data

## Install fiftyone 

In [ ]:
#   For windows:
#        pip3 install fiftyone --user

## Load coco dataset (ground-truth)

**Note**: 

(1) the folder/path of images in json file must contains " / " instead of " \ " 

(2) In json file, "image_id" and the "id" for annotations must not start "0", otherwise the results will be not correct

In [1]:
import fiftyone as fo

# A name for the dataset
# name = "my-dataset"

# GJO test dataset
image_path = r"U:\AIMMW\GJO\Auto\10%_images_test\tiles_224\images"
labels_path = r"U:\AIMMW\GJO\Auto\10%_images_test\tiles_224\labels_coco\SL_test_2620.json"

# Amsterdam test dataset
# image_path = r"U:\AIMMW\GJO\Exp4\Amsterdam\images"
# labels_path = r"U:\AIMMW\GJO\Exp4\Amsterdam\labels_coco\test.json"

# Groningen test dataset
# image_path = r"U:\AIMMW\GJO\Exp4\Groningen\test\tiles_224\images"
# labels_path = r"U:\AIMMW\GJO\Exp4\Groningen\test\tiles_224\labels_coco\SL_test_525.json"

# Vietnam test dataset
# image_path = r"U:\AIMMW\GJO\Exp4\Vietnam_3_class_V5\test\tiles_224\images"
# labels_path = r"U:\AIMMW\GJO\Exp4\Vietnam_3_class_V5\test\tiles_224\labels_coco\SL_test_1091.json"


# The type of the dataset being imported
dataset_type = fo.types.COCODetectionDataset  # coco format

dataset = fo.Dataset.from_dir(
    dataset_type=dataset_type,
    data_path=image_path,
    labels_path=labels_path,
    # include_id=True,
    # name=name,
)
# print(dataset)

 100% |███████████████| 1849/1849 [21.1s elapsed, 0s remaining, 135.9 samples/s]      


In [ ]:
# show the first data in the dataset,
# Carefully check the "label" name!
# Note the coordinates of the bbox is scaled into [0,1]

# print(dataset.first().detections.detections[0])

## Load predictions

When inferencing a model on a test dataset using Detectron2, it will output a prediction file named "coco_instances_results.json" with predicted bbox

In [3]:
import fiftyone.utils.coco as fouc
import json

#
# Mock COCO predictions, where:
# - `image_id` corresponds to the `coco_id` field of `coco_dataset`
# - `category_id` corresponds to classes in `coco_dataset.default_classes`
#
# predictions = [
#     {"image_id": 1, "category_id": 18, "bbox": [258, 41, 348, 243], "score": 0.87},
#     {"image_id": 2, "category_id": 11, "bbox": [61, 22, 504, 609], "score": 0.95},
# ]

json_file= r"U:\AIMMW\Paper 3 datasest and results\checkpoints_(backup_2024.2.7)\train_weights\SSL_models_exp3\Freeze_4\SW_RN50_100e_FTAL\SSL_train_2628\test_results\coco_instances_results.json"

with open(json_file, 'r') as fcc_file:
    fcc_data = json.load(fcc_file)
# print(fcc_data)
    
predictions = fcc_data


# Add COCO predictions to `predictions` field of dataset
classes = dataset.default_classes
print("classes:", classes)
# classes = ['hold_space', 'litter',]
# classes = ['airplane', 'apple', ...]
fouc.add_coco_labels(dataset, "predictions", predictions, classes)

# Verify that predictions were added to two images
print(dataset.count("predictions"))  # 2

classes: ['0', 'litter']
1620


In [4]:
print(dataset.count("detections"))  # the number of images

1849


In [ ]:
# show the first data (with prediction bbox) in the dataset ,
# Carefully check the "label" name!
# Note the coordinates of the predicted bbox is scaled into [0,1]

# print(dataset.first())

In [ ]:
# sample = dataset.first()
# print(sample.detections.detections[0])

## Output results 

by comparing ground-truth labels and predictions bbox

**Note**: Predicted and ground truth objects are matched using a specified IoU threshold (default = 0.50). This threshold can be customized via the iou parameter. The IoU selection affects the calculation of TP, FN, are FP value.

In [6]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

# Evaluate the objects in the `predictions` field with respect to the
# objects in the `ground_truth` field
results = dataset.evaluate_detections(
    "predictions",
    gt_field="detections",
    iou=0.5,
    method="coco",   
    eval_key="eval",
    compute_mAP=True,
)

Evaluating detections...
 100% |███████████████| 1849/1849 [46.9s elapsed, 0s remaining, 67.0 samples/s]      
Performing IoU sweep...
 100% |███████████████| 1849/1849 [39.2s elapsed, 0s remaining, 63.6 samples/s]      


In [7]:
results

### mAP

Note: this is the mAP at IoU=.50:.05:.95

In [ ]:
print(results.mAP())

### TP, FP, FN

In [ ]:
# Print some statistics about the total TP/FP/FN counts
print("TP: %d" % dataset.sum("eval_tp"))
print("FP: %d" % dataset.sum("eval_fp"))
print("FN: %d" % dataset.sum("eval_fn"))

### Precision, recall, F1-score

In [9]:
classes = 1
counts = dataset.count_values("detections.detections.label")
classes = sorted(counts, key=counts.get, reverse=True)[:classes]

# Print a classification report for the top-10 classes
results.print_report(classes=classes)

              precision    recall  f1-score   support

      litter       0.47      0.68      0.55      2620

   micro avg       0.47      0.68      0.55      2620
   macro avg       0.47      0.68      0.55      2620
weighted avg       0.47      0.68      0.55      2620



### Precision-recall curve

In [ ]:
plot = results.plot_pr_curves(classes=["litter"])
plot.show()

### Confusion matrix

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz


# Perform evaluation, allowing objects to be matched between classes
results = dataset.evaluate_detections(
    "predictions",
    gt_field="detections",
    method="coco",
    classwise=False,
)

# Generate a confusion matrix for the specified classes
plot = results.plot_confusion_matrix(classes=["litter"])
plot.show()